In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [5]:
PROJECT_PATH = '../'
DATA_PATH = '/scratch/ceph/swei20/data/ae/dataset/test'
sys.path.insert(0, PROJECT_PATH)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import tensorflow as tf
# tf.enable_v2_behavior()
gpus = tf.config.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading Data

In [27]:
with h5py.File(fn, 'r') as f:
    flux10kmm = f['flux10kmm'][()]
#     flux10kmm = f['flux10kmm'][()]

In [31]:
b = a,a

In [33]:
len(b)

2

In [30]:
a, a == a, a

(array([[1, 2, 3],
        [4, 5, 6]]),
 array([[ True,  True,  True],
        [ True,  True,  True]]),
 array([[1, 2, 3],
        [4, 5, 6]]))

In [29]:
a

array([[1, 2, 3],
       [4, 5, 6]])

## AE

In [8]:
from ae.models.ae_model import AE

In [9]:
input_dim, latent_dim = 4096, 32
hidden_units = []
# reg1 = 1e-5
reg1 = None
encoder_dp=0.2
lr = 0.001
loss='mae'

In [13]:
model= AE(input_dim, latent_dim, hidden_units, reg1=reg1, encoder_dp=encoder_dp, loss=loss, lr=lr)

[4096, 32]
Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spec (InputLayer)            [(None, 4096)]            0         
_________________________________________________________________
encoder (Functional)         (None, 32)                131104    
_________________________________________________________________
decoder (Functional)         (None, 4096)              135168    
Total params: 266,272
Trainable params: 266,272
Non-trainable params: 0
_________________________________________________________________


In [20]:
x_train = flux10kmm
model.fit(x_train, ep=10)

Epoch 1/10
563/563 [==============================] - 2s 3ms/step - loss: 0.0148 - val_loss: 0.0074
Epoch 2/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0091 - val_loss: 0.0093
Epoch 3/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0079 - val_loss: 0.0049
Epoch 4/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0072 - val_loss: 0.0072
Epoch 5/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0069 - val_loss: 0.0039
Epoch 6/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0071 - val_loss: 0.0044
Epoch 7/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0070 - val_loss: 0.0043
Epoch 8/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0067 - val_loss: 0.0045
Epoch 9/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0070 - val_loss: 0.0050
Epoch 10/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0068 - val_loss: 0.0040